# Chapter 9 Gauss Elimination

In [1]:
import numpy as np

In [2]:
def gauss_naive(a, b):
    """
    a x = b   a.shape = (n, n), b.shape = (n, 1)
    """
    #a = np.array(a, dtype=np.float64)
    #b = np.array(b, dtype=np.float64)
#     b = b.copy()
    n = len(b)
    # forward elimination
    for k in range(n - 1):
        for i in range(k+1, n):
            f = a[i, k]/a[k, k]
            a[i, k+1:] -= f*a[k, k+1:]
            b[i] -= f*b[k]
    # backward substitution    
    for k in reversed(range(n)):
        b[k] = (b[k] - np.dot(a[k, k+1:], b[k+1:]))/a[k, k]
    return b

In [4]:
# Example 9.3
A = np.array([[3, -0.1, -0.2], [0.1, 7, -0.3], [0.3, -0.2, 10]])
b = np.array([7.85, -19.3, 71.4])
gauss_naive(A, b)

array([ 3. , -2.5,  7. ])

In [13]:
# def simp13(f, h):
#     """ n = len(f) must be odd """
#     s = h/3*(f[0] + f[-1] + 4*np.sum(f[1:-1:2]) + 2*np.sum(f[2:-1:2]))
#     return s

def _sim13(f, h):
       
        f0, f1 = f[::2], f[1::2]
        return np.sum(h/3*(f0[:-1] + 4*f1 + f0[1:]))
    
   
    if n % 2 == 1:
        return _sim13(f, h)
    else:
        s = _sim13(f[:n-3], h)
        f0, f1, f2, f3 = f[-4:]
        return s + 3*h*(f0 + 3*f1 + 3*f2 + f3)/8
    
def fun(x):
    return np.sin(x)

a, b = 0, np.pi
n = 21
x = np.linspace(a, b, n)
h = (b - a)/(n - 1)
f = fun(x)

I2 = _sim13(f, h)
print(I2)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 12)

In [4]:
n = 4
a = np.empty((n,n))
b = np.empty(n)
        
for i in range(n):
    b[i] = ((i + 2.)**n - 1.)/(i + 1)
    for j in range(n):
        a[i,j] = (i + 2.)**j
        
x = gauss_naive(a, b)        
x

array([1., 1., 1., 1.])

### Pivoting

In [5]:
def gauss_pivot(a, b):
    a = a.copy()
    b = b.copy()
    n = len(b)
    # forward elimination
    for k in range(n - 1):
        
        # partial pivot
        i = k + np.argmax(a[k:, k])
        a[[k, i]] = a[[i, k]]
        b[[k, i]] = b[[i, k]]        
        
        for i in range(k + 1, n):
            factor = a[i, k] / a[k, k]
            a[i, k+1:] -= factor * a[k, k+1:]
            b[i] -= factor * b[k]
    # backward substitution    
    for k in reversed(range(n)):
        b[k] = (b[k] - np.dot(a[k, k+1:], b[k+1:])) / a[k, k]
    return b

In [6]:
a = np.array([[0, -0.1, -0.2], [0.1, 7, -0.3], [0.3, -0.2, 10]])
b = np.array([7.85, -19.3, 71.4])
x1 = np.linalg.solve(a, b)
x2 = gauss_pivot(a, b)
print(x1, x2)

[1183.97560976  -20.90525328  -28.79737336] [1183.97560976  -20.90525328  -28.79737336]


### Tridiagonal System

In [7]:
def tdma(tri):
    # tri-diagonal system
    tri = np.copy(tri)
    e, f, g, r = tri.T
    n = len(f)
    for k in range(1, n):
        factor = e[k] / f[k - 1]
        f[k] -= factor * g[k - 1]
        r[k] -= factor * r[k - 1]
    x = np.zeros_like(f)
    x[-1] = r[-1] / f[-1]
    for k in reversed(range(n-1)):
        x[k] = (r[k] - g[k] * x[k+1]) / f[k]
    return x

In [8]:
# example 9.5
a = np.array([[0, 2.04, -1], [-1, 2.04, -1], [-1, 2.04, -1], [-1, 2.04, 0]])
b = np.array([40.8, 0.8, 0.8, 200.8])
# tri=np.hstack([a, b[:, np.newaxis]])
tri = np.c_[a, b[:, np.newaxis]]
tdma(tri)

array([ 65.96983437,  93.77846211, 124.53822833, 159.47952369])